# Demo: Navie - CNN


- This is a four layers Navie-CNN demo. 
- Training on CIFAR10, define a dataset transformation function which contains Decrete Fourier Transform function.
- But the applied path of DFT haven'd conformed yet, which will be discussed in later paper work.

In [1]:
# Run bunch of settings
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms # Provide common datasets and transformation function.
print("PyTorch Version: ",torch.__version__)

PyTorch Version:  1.4.0


In [2]:
# Define a classical CNN model
# The input of model is 5000 * 3 * 32 * 32 and label's vector is 5000 * 1
# Each image's scale is 3 * 32 * 32
class Net(nn.Module):
    def __init__(self):
        # nn.Conv2d(depth, num_init_features, kernel_size, stride, padding, bias=False)
        # nn.Linear(num_input, num_output)
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 1, 1)      # (32 - 1 + 0) / 1 + 1 = 32
        self.conv2 = nn.Conv2d(64, 128, 4, 1, 3) # (32 - 4 + 3) / 1 + 1 = 32 
        self.conv3 = nn.Conv2d(128, 256, 5, 1)   # (32 - 5 + 0) / 1 + 1 = 28 -> 20 * 28 * 28
        self.conv4 = nn.Conv2d(256, 256, 5, 1)   # (28 - 5 + 0) / 1 + 1 = 24 -> 50 * 20 * 20
        self.fc1 = nn.Linear(5*5*256, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):     #x: #   1 *  32  *  32 
        x = F.relu(self.conv1(x)) #  64 *  32  *  32
        x = F.max_pool2d(x,2,1)   #  64 *  32  *  32
        x = F.relu(self.conv2(x)) # 128 *  32  *  32
        x = F.max_pool2d(x,2,1)   # 128 *  32  *  32
        x = F.relu(self.conv3(x)) #  20 *  28  *  28
        x = F.max_pool2d(x,2,2)   #  20 *  14  *  14
        x = F.relu(self.conv4(x)) #  50 *  10  *  10
        x = F.max_pool2d(x,2,2)   #  50 *   5  *   5
        x = x.view(-1, 5*5*256)    # reshape (5 * 2 * 10), view(5, 20) -> (5 * 20)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        #return x
        return F.log_softmax(x, dim=1) # log probability


In [8]:
# Defin the train function 
# input : model, device, train_loader, optimizer, num_epoch
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        pred = model(data) # batch_size * classes -> 5 * 10 
        loss = F.cross_entropy(pred, target) + F.nll_loss(pred, target)  
        
        #print("pred.shape :{}".format(pred.shape))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if idx % 1000 == 0:
            print("Train Epoch: {}, iteration: {}, Loss: {}".format(
                epoch, idx, loss.item()))
            

In [4]:
# Defin the test function 
# input : model, device, test_loader
def test(model, device, test_loader):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, (data, target) in enumerate(test_dataloader):
            data, target = data.to(device), target.to(device)
            
            output = model(data)
            total_loss += F.nll_loss(output, target, reduction="sum").item() 
            pred = output.argmax(dim=1) # batch_size  -> 5 
            #print("pred.shape :{}".format(pred.shape))
            correct += pred.eq(target.view_as(pred)).sum().item()
            #print("idx :{} Correct :{}".format(idx, correct))
            
    total_loss /= len(test_dataloader.dataset)    
    acc = correct / len(test_dataloader.dataset) * 100.
    print("Test loss:{}, Accuracy:{}".format(total_loss, acc))

In [5]:
# Define transformation function
class ToArray(object):
    def __call__(self, pic):
        return np.array(pic)

    def __repr__(self):
        return self.__class__.__name__ + '()'

In [6]:
# Define transformation function
class Fourier(object):
    def __call__(self, pic):
        x,y = 32, 32
        cw = 6
        # Normal Gaussion Filter
        filter = np.ones((y,x))
        filter[int(y/2 - cw):int(y/2 + cw), int(x/2 -cw):int(x/2 + cw)] = 0 
        # Deceret Fourier Tranformation
        fshift = np.fft.fftshift(np.fft.fft2(pic))
        res1 = np.log(1 + np.abs(fshift)) 
        fshift *= filter      
        res2 = np.log(1 + np.abs(fshift))    
        iimg = np.fft.ifft2(np.fft.ifftshift(fshift))
        iimg = np.abs(iimg)
        a = np.ones((x, y, 3))
        for i in range(3):
            a[:,:,i] = iimg
        a = np.maximum(0, a / np.max(a)).astype(np.float32)
        return a

    def __repr__(self):
        return self.__class__.__name__ + '()'

In [59]:
# Extract mean and std of train data.
data = [d[0].data.cpu().numpy() for d in data]
avg_train = np.mean(data_train)
std_train = np.std(data_train)
print("avg_train :{}, std_train :{}.".format(avg_train, std_train))

avg_train :0.4733648896217346, std_train :0.25156906247138977.


In [7]:
# Training Device and dataloader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 5

train_dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./CIFAR10_data", train=True, download=True,
                                    transform=transforms.Compose([transforms.Grayscale(), 
                                                                  Fourier(),
                                                                  transforms.ToTensor(),
                                                                  transforms.Normalize((0.4734,), (0.2516,))
                                                                 ]))
    , batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True
)
test_dataloader = torch.utils.data.DataLoader(
    datasets.CIFAR10("./CIFAR10_data", train=False, download=True,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.4734,), (0.2516,))
                     ]))
    , batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True
)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
# Training Procress
lr = 1e-6
momentum = 0.5
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)

num_epochs = 5
for epoch in range(num_epochs):
    train(model, device, train_dataloader, optimizer, epoch)
    test(model, device, test_dataloader)
torch.save(model.state_dict(), "CIFAR10_cnn.pt")

Train Epoch: 0, iteration: 0, Loss: 4.612001419067383
Train Epoch: 0, iteration: 1000, Loss: 4.5965681076049805
Train Epoch: 0, iteration: 2000, Loss: 4.628288269042969
Train Epoch: 0, iteration: 3000, Loss: 4.459005832672119
Train Epoch: 0, iteration: 4000, Loss: 4.6137495040893555
Train Epoch: 0, iteration: 5000, Loss: 4.898815155029297
Train Epoch: 0, iteration: 6000, Loss: 4.197612762451172
Train Epoch: 0, iteration: 7000, Loss: 4.358684539794922
Train Epoch: 0, iteration: 8000, Loss: 3.7393100261688232
Train Epoch: 0, iteration: 9000, Loss: 3.9017131328582764
Test loss:2.839319881296158, Accuracy:11.5
Train Epoch: 1, iteration: 0, Loss: 4.358511924743652
Train Epoch: 1, iteration: 1000, Loss: 4.090919494628906
Train Epoch: 1, iteration: 2000, Loss: 4.677031993865967
Train Epoch: 1, iteration: 3000, Loss: 3.9711737632751465
Train Epoch: 1, iteration: 4000, Loss: 5.256221771240234
Train Epoch: 1, iteration: 5000, Loss: 4.568746089935303
Train Epoch: 1, iteration: 6000, Loss: 4.45854